In [4]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_model
from botorch.acquisition.multi_objective import qExpectedHypervolumeImprovement

from botorch.optim import optimize_acqf
from botorch.utils.multi_objective.box_decompositions import NondominatedPartitioning
from botorch.utils.multi_objective.pareto import is_non_dominated
# standard scaler
from sklearn.preprocessing import StandardScaler
# minmax scaler
from sklearn.preprocessing import MinMaxScaler
# Define your custom loss function

from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from botorch.utils.transforms import *
import torch
import botorch
# Plotting imports and initialization
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.plot.pareto_utils import compute_posterior_pareto_frontier
from ax.plot.pareto_frontier import plot_pareto_frontier


In [14]:
#defining everything for dummy testing

param_bounds = {
    'c1': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'W', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c2': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'K', 'unit': 'MPa', 'type': 'yielding'}, 
    'c3': {'lowerBound': 0, 'upperBound': 1, 'exponent': 0.1, 'name': 'e0', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c4': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1.0, 'name': 'n', 'unit': 'dimensionless', 'type': 'hardening'}, 
    'c5': {'lowerBound': 0, 'upperBound': 2, 'exponent': 1000.0, 'name': 'sigma_y', 'unit': 'MPa', 'type': 'yielding'}, 
    'c6': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'sigma_sat', 'unit': 'MPa', 'type': 'hardening'}, 
    'c7': {'lowerBound': 0, 'upperBound': 1, 'exponent': 1000.0, 'name': 'b', 'unit': 'dimensionless', 'type': 'hardening'}
}


geometries = ['NDBR50', 'NDBR6', 'CHD6']

def calculate_loss(simulated_curve, target_curve):
    # This is a placeholder. Replace this with your actual loss function.
    return np.mean(np.sum((simulated_curve - target_curve)**2))


def generate_dummy_target_curves():
    geometries = ['NDBR50', 'NDBR6', 'CHD6']
    targetCurves = {geometry: {'force': np.random.rand(100), 'displacement': np.linspace(0, 2, 100)} for geometry in geometries}
    return targetCurves

def generate_dummy_combined_data():
    geometries = ['NDBR50', 'NDBR6', 'CHD6']
    params = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7']
    combined_data = {
        tuple((param, np.random.rand()) for param in params): 
        {geometry: {'force': np.random.rand(100), 'displacement': np.linspace(0, 2, 100)} for geometry in geometries}
        for _ in range(100)
    }
    return combined_data


import numpy as np

def scale_dataset(data):
    # Step 1: Calculate min and max values along each dimension
    min_vals = np.min(data, axis=0)
    max_vals = np.max(data, axis=0)

    # Step 2: Perform min-max scaling
    scaled_data = (data - min_vals) / (max_vals - min_vals)

    # Step 3: Calculate means along each dimension
    mean_vals = np.mean(scaled_data, axis=0)

    # Step 4: Perform mean normalization
    normalized_data = scaled_data - mean_vals

    return normalized_data


In [20]:
# Generate dummy data
combined_interpolated_param_to_geom_FD_Curves_smooth = np.load('combined_interpolated_param_to_geom_FD_Curves_smooth.npy', allow_pickle=True).tolist()

targetCurves = np.load('targetCurves.npy', allow_pickle=True).tolist()

# Convert your data to the required format
params = []
losses = []
for param_tuple, geom_dict in combined_interpolated_param_to_geom_FD_Curves_smooth.items():
    params.append([value for param, value in param_tuple])
    losses.append([calculate_loss(geom_dict[geometry]["force"], targetCurves[geometry]["force"]) for geometry in geometries])

std_scaler = StandardScaler()
minmax_scaler = MinMaxScaler()

# Normalize the parameters
params = torch.tensor(params, dtype=torch.float64)
#scaled_params = std_scaler.fit_transform(params)
scaled_params = minmax_scaler.fit_transform(params)
scaled_params = torch.tensor(scaled_params, dtype=torch.float64)

# losses = torch.tensor(losses, dtype=torch.float64)
#scaled_losses = std_scaler.fit_transform(losses)
scaled_losses = minmax_scaler.fit_transform(losses)
print(np.std(scaled_losses, axis=0))
scaled_losses = torch.tensor(scaled_losses, dtype=torch.float64)

# print(params.shape)
# (200, 7)
# print(losses.shape)
# (200, 3)

# Train a GP model

gp = SingleTaskGP(train_X=scaled_params, train_Y=scaled_losses)


mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

# Define the bounds of the search space
lower_bounds = torch.tensor([param_bounds[param]['lowerBound'] for param in param_bounds.keys()]).float()
upper_bounds = torch.tensor([param_bounds[param]['upperBound'] for param in param_bounds.keys()]).float()

bounds = torch.stack([lower_bounds, upper_bounds])

# Define the reference point as slightly worse than the worst observed values
ref_point = torch.zeros(scaled_losses.shape[1])

# Define the partitioning of the output space
partitioning = NondominatedPartitioning(ref_point=ref_point, Y=scaled_losses)

# Define the acquisition function
acq_func = qExpectedHypervolumeImprovement(
    model=gp, 
    ref_point=torch.zeros(scaled_losses.shape[1]),
    partitioning=partitioning,
)

# acq_func = q

# Optimize the acquisition function to find the next set of parameters to evaluate
candidates, _ = optimize_acqf(
    acq_function=acq_func,
    bounds=bounds,
    q=10,  # change this to the number of candidates you want to generate
    num_restarts=5,
    raw_samples=10,
)

# Denormalize the candidates
# next_params = minmax_scaler.inverse_transform(candidates.detach().numpy())
next_params = std_scaler.inverse_transform(candidates.detach().numpy())

# Convert the tensor to a list of dictionaries
next_param_dicts = [{param: value.item() for param, value in zip(param_bounds.keys(), next_param)} for next_param in next_params]

# Select the non-dominated solutions
pareto_mask = is_non_dominated(torch.stack([gp.posterior(next_param.unsqueeze(0)).mean for next_param in next_params]))
pareto_solutions = [next_param_dicts[i] for i in range(len(next_param_dicts)) if pareto_mask[i]]

print(pareto_solutions)



[0.15221789 0.15649037 0.16322542]


c:\Users\springnuance\AppData\Local\Programs\Python\Python311\Lib\site-packages\botorch\models\utils\assorted.py:201: InputDataWarning:

Input data is not standardized. Please consider scaling the input to zero mean and unit variance.



KeyboardInterrupt: 

In [ ]:
# InputDataWarning:

# Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
params_copy = params.copy()
params = torch.tensor(params, dtype=torch.float64)
# Input data is not contained to the unit cube. Please consider min-max scaling the input data.
# scaled_params = MinMaxScaler().fit_transform(params)
# scaled_params = StandardScaler().fit_transform(params)

bounds = [[-0.5 for _ in range(params.shape[1])], [0.5 for _ in range(params.shape[1])]]
bounds = torch.tensor(bounds, dtype=torch.float64)
# print(bounds.shape)

# scaled_params = normalize(params, bounds=bounds)
scaled_params = standardize(params)
scaled_params = normalize(scaled_params, bounds=bounds)
losses = torch.tensor(losses, dtype=torch.float64)

#scaled_params = torch.tensor(scaled_params, dtype=torch.float64)
bounds = [[-0.5 for _ in range(losses.shape[1])], [0.5 for _ in range(losses.shape[1])]]
bounds = torch.tensor(bounds, dtype=torch.float64)

#scaled_losses = MinMaxScaler().fit_transform(losses)
#scaled_losses = StandardScaler().fit_transform(scaled_losses)
#scaled_losses = torch.tensor(scaled_losses, dtype=torch.float64)

#scaled_losses = normalize(losses, bounds=bounds)
scaled_losses = standardize(losses)
scaled_losses = normalize(scaled_losses, bounds=bounds)